In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather
import glob
import os
import pyarrow
from xclim import ensembles as ens
from xclim import subset

In [2]:
import matplotlib.pyplot as plt

In [3]:
folder = "/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/"
out_folder = "/home/mlopez/EXEC/TimeMeansModels/"

# Select variable

In [38]:
#Variable 
#var="tg_mean"
#var ="tn_mean"
#var ="tx_mean"
#var ="solidprcptot"
var ="prcptot"
#var ="dlyfrzthw"
#var ="growing_season_length"
#var ="DJC"


files85 = glob.glob(folder+"*"+"rcp85_"+var+"_annual.nc")
files45 = glob.glob(folder+"*"+"rcp45_"+var+"_annual.nc")


filesSeas45 =  glob.glob(folder+"*rcp45_"+var+"_seasonal.nc")
filesSeas85 =  glob.glob(folder+"*rcp85_"+var+"_seasonal.nc")



# FOR RCP 45

In [46]:
dsEns45= ens.create_ensemble(files45)
#dsEns45

## Getting time windows and percentiles

In [48]:
# Table 
# Mapping data - 30 year means
tmp1 = dsEns45.sel(time=(dsEns45.time.dt.year>=1981))
window = 30
time1 = tmp1.time[0::window]
# 30 y means
ds30yavgTable = tmp1.coarsen(time=window).mean()
ds30yavgTable['time'] = time1
# spatial mean
perc30yavgTable = ens.ensemble_percentiles(ds30yavgTable)
perc30yavgTable 

<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416, realization: 11, time: 4)
Coordinates:
  * lon          (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * time         (time) datetime64[ns] 1981-01-01 2011-01-01 ... 2071-01-01
  * lat          (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * realization  (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    prcptot_p10  (time, lat, lon) float32 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>
    prcptot_p50  (time, lat, lon) float32 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>
    prcptot_p90  (time, lat, lon) float32 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>

In [50]:
%time dfperT = perc30yavgTable.drop('realization').to_dataframe().dropna()

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)



CPU times: user 51.1 s, sys: 4min 17s, total: 5min 8s
Wall time: 4.99 s


## Split by periodos

In [54]:
df45T1980 = dfperT.query("time=='1981-01-01'")
df45T2041= dfperT.query("time=='2041-01-01'")
df45T2071 = dfperT.query("time=='2071-01-01'")

In [58]:
listdf45p = [df45T1980, df45T2041, df45T2071 ]

## TODO
- Cut by region
- get the mean by region
- Do the same for rcp84 and seasonal 45 and 85. 
- Put all together into the table

## Seasonal

In [42]:
dsEnsSeas45 = ens.create_ensemble(filesSeas45)

## To split by region

In [55]:
short_dict = {"DDE_STF_20K_REG_FOR_VUE_S": ("NM_REG_FOR", "RF"), 
            "DDE_STF_20K_UA_PER_VUE_S": ("PER_NO_UA", "UA"), 
              "DOM_BIO": ("NOM", "DB"), 
              "REG_ECO": ("NOM", "RE"), 
              "SDOM_BIO": ("NOM", "SDB"), 
              "Secteurs_Operations_Regionales": ("D_GENERAL", "SOR"), 
              "SREG_ECO": ("NOM", "SRE"),  
              "territoire_guide": ("TER_GUIDE", "TG") 
              }

for region, (name, short) in short_dict.items():
    print(region, name, short)

DDE_STF_20K_REG_FOR_VUE_S NM_REG_FOR RF
DDE_STF_20K_UA_PER_VUE_S PER_NO_UA UA
DOM_BIO NOM DB
REG_ECO NOM RE
SDOM_BIO NOM SDB
Secteurs_Operations_Regionales D_GENERAL SOR
SREG_ECO NOM SRE
territoire_guide TER_GUIDE TG


In [56]:
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [57]:
d = { 'BAS-SAINT-LAURENT':"Bas-Saint-Laurent", 
     'SAGUENAY -LAC-SAINT-JEAN': "Saguenay -Lac-Saint-Jean",
     'CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES':"Capitale-Nationale-Chaudiere-Appalaches",
     'MAURICIE-CENTRE-DU-QUÉBEC':'Mauricie-Centre-du-Quebec','OUTAOUAIS':'Outaouais', 
     'ABITIBI-TEMISCAMINGUE':'Abitibi-Temiscamingue', 'COTE-NORD':'Cote-Nord',
     'NORD-DU-QUEBEC':'Nord-Du-Quebec', 'GASPESIE-ILES-DE-LA-MADELEINE':'Gaspesie-Iles-De-La-Madeleine', 
     'LANAUDIERE':'Lanaudiere','LAURENTIDES':"Laurentides", 
     "ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL":"Estrie-Monteregie-Laval-Montreal", 
     "é": "e", "É": "E", "à": "a", "è": "e", "Î": "i", "È": "E", "ô" : "o", "Ç":"C", "ç":"c",
       } 

In [89]:
for region, (name, short) in short_dict.items():
    print("------------------------------------------------------------------")
    print ("1/8... Cutting by region")
    print(region, name)
    dfregion = pd.read_feather("/home/mlopez/EXEC/Grids-polygons-regions/"+region+"lat_lon.feather")
    print ("2/8... Merging region with data")
    dfdatareg45 = pd.merge(dfregion, dfperT.reset_index(), on=["lat","lon"])
    print ("3/8... Get mean by year for every sub-region")
    #print(dfmean45UA)
    dfmean45UA = dfdatareg45.groupby(["time", name]).mean()
    dfmean45UA2 = dfmean45UA.reset_index()
    print ("4/8... Round")
    if (var == "tg_mean" or var== "tn_mean" or var== "tx_mean"):
        dfmean45UA2[var+"_p10"] = dfmean45UA2[var+"_p10"]- 273.15
        dfmean45UA2[var+"_p50"] = dfmean45UA2[var+"_p10"]- 273.15
        dfmean45UA2[var+"_p90"] = dfmean45UA2[var+"_p10"]- 273.15
    if (var == "solidprcptot"):
        dfmean45UA2[var+"_p10"] = dfmean45UA2[var+"_p10"]/10
        dfmean45UA2[var+"_p50"] = dfmean45UA2[var+"_p50"]/10
        dfmean45UA2[var+"_p90"] = dfmean45UA2[var+"_p90"]/10
    dfmean45UA2[var+"_p10"] = round(dfmean45UA2[var+"_p10"], 2)
    dfmean45UA2[var+"_p50"] = round(dfmean45UA2[var+"_p50"], 2)
    dfmean45UA2[var+"_p90"] = round(dfmean45UA2[var+"_p90"], 2)
    dfmean45UA2[var+"_p10_p90"] = dfmean45UA2[var+"_p10"].map(str)+" - "+dfmean45UA2[var+"_p90"].map(str)
    dfreg45 = df1 = dfmean45UA2[["time", name, var+"_p50", var+"_p10_p90"]]
    print (dfreg45)


------------------------------------------------------------------
1/8... Cutting by region
DDE_STF_20K_REG_FOR_VUE_S NM_REG_FOR
2/8... Merging region with data
3/8... Get mean by year for every sub-region
4/8... Round
         time                               NM_REG_FOR  prcptot_p50  \
0  1981-01-01                    ABITIBI-TEMISCAMINGUE       914.39   
1  1981-01-01                        BAS-SAINT-LAURENT      1048.18   
2  1981-01-01  CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES      1166.48   
3  1981-01-01                                COTE-NORD      1009.41   
4  1981-01-01         ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL      1098.09   
5  1981-01-01            GASPESIE-ILES-DE-LA-MADELEINE      1093.05   
6  1981-01-01                               LANAUDIERE      1043.85   
7  1981-01-01                              LAURENTIDES      1028.89   
8  1981-01-01                MAURICIE-CENTRE-DU-QUÉBEC      1004.67   
9  1981-01-01                           NORD-DU-QUEBEC       822.00   


3/8... Get mean by year for every sub-region
4/8... Round
          time                                                NOM  \
0   1981-01-01              Collines ceinturant le lac Saint-Jean   
1   1981-01-01  Collines de Havre-Saint-Pierre et de Blanc-Sablon   
2   1981-01-01        Collines de l'Outaouais et du Témiscamingue   
3   1981-01-01                      Collines de la basse Gatineau   
4   1981-01-01                   Collines des moyennes Appalaches   
..         ...                                                ...   
183 2071-01-01            Plaine du lac Saint-Jean et du Saguenay   
184 2071-01-01                   Plaines et coteaux du lac Simard   
185 2071-01-01                                         Île Mingan   
186 2071-01-01                                    Île d'Anticosti   
187 2071-01-01                               Îles-de-la-Madeleine   

     prcptot_p50    prcptot_p10_p90  
0         994.33   977.21 - 1009.14  
1        1075.00  1059.51 - 1091.98  

3/8... Get mean by year for every sub-region
4/8... Round
          time                                              NOM  prcptot_p50  \
0   1981-01-01              Collines de Saint-Jérôme-Grand-Mère      1068.14   
1   1981-01-01                    Collines de la basse Gatineau       979.03   
2   1981-01-01                 Collines de la rivière Vermillon       985.51   
3   1981-01-01  Collines des lacs Musquanousse et du Vieux Fort      1094.59   
4   1981-01-01                 Collines du Grand-Lac-Bostonnais      1076.19   
..         ...                                              ...          ...   
263 2071-01-01                           Plaine du lac Matagami       901.92   
264 2071-01-01                           Plaine du lac Opémisca      1008.08   
265 2071-01-01          Plaine du lac Saint-Jean et du Saguenay      1022.43   
266 2071-01-01                 Plaines et coteaux du lac Simard       949.93   
267 2071-01-01                             Îles-de-la-Madelein

In [87]:
dfmean45UA2.prcptot_p10.map(str)

0      969.76
1      957.29
2     1063.35
3     1142.27
4      946.92
       ...   
95    1062.41
96    1098.81
97    1011.58
98    1144.77
99    1034.46
Name: prcptot_p10, Length: 100, dtype: object

In [82]:
var

'prcptot'

# Creating ROWS

In [19]:
col1 = ["Annuel","Percentiles (10 - 90)","Hiver","Percentiles (10 - 90)","Printemps","Percentiles (10 - 90)",
        "Ete","Percentiles (10 - 90)","Automne","Percentiles (10 - 90)"]

# Creating COLUMNS

In [ ]:
dfTable = pd.DataFrame()
dfTable["Saison"] = col1

In [44]:
#dfTable

## Adding columns for RCP85

In [78]:
dfreg45

,time,TER_GUIDE,prcptot_p10,prcptot_p50,prcptot_p90
0,1981-01-01,1a,969.76,993.74,1002.86
1,1981-01-01,2a,957.29,979.45,988.19
2,1981-01-01,2b,1063.35,1081.70,1091.75
3,1981-01-01,2c,1142.27,1161.06,1179.84
4,1981-01-01,3ab,946.92,961.13,975.57
...,...,...,...,...,...
95,2071-01-01,6hi,1062.41,1110.66,1174.09
96,2071-01-01,6j,1098.81,1138.51,1185.91
97,2071-01-01,6kl,1011.58,1048.38,1129.31
98,2071-01-01,6mn,1144.77,1185.44,1222.40


In [33]:
dfTable["1981-2010"] = [round(dfReg1T1980.tg_mean_p50.values[0]-273.15, 2), (str(round(dfReg1T1980.tg_mean_p10.values[0]-273.15,2))
                                                            +" - "+str(round(dfReg1T1980.tg_mean_p90.values[0]-273.15,2))),str(1)+"-"+str(99),2,3,4,5,6,7,8]
dfTable["2041-2070"] = [round(dfReg1T2041.tg_mean_p50.values[0]-273.15, 2), (str(round(dfReg1T2041.tg_mean_p10.values[0]-273.15,2))
                                                            +" - "+str(round(dfReg1T2041.tg_mean_p90.values[0]-273.15,2))),str(1)+"-"+str(99),2,3,4,5,6,7,8]
dfTable["2071-2100"] = [round(dfReg1T2071.tg_mean_p50.values[0]-273.15, 2), (str(round(dfReg1T2071.tg_mean_p10.values[0]-273.15,2))
                                                            +" - "+str(round(dfReg1T2071.tg_mean_p90.values[0]-273.15,2))),str(1)+"-"+str(99),2,3,4,5,6,7,8]
dft2 = dfTable.set_index("Saison")

In [29]:
dft2

,1981-2010,2041-2070,2071-2100
Saison,,,
Annuel,2.82903,5.95568,8.71639
Percentiles (10 - 90),2.79 - 3.03,5.31 - 6.9,6.5 - 9.8
Hiver,1-99,1-99,1-99
Percentiles (10 - 90),2,2,2
Printemps,3,3,3
Percentiles (10 - 90),4,4,4
Ete,5,5,5
Percentiles (10 - 90),6,6,6
Automne,7,7,7


In [24]:
dft2.to_csv('dfTable.csv')